[View in Colaboratory](https://colab.research.google.com/github/alvarogutyerrez/alvarogutyerrez/blob/master/S03_data_handle.ipynb)

# Business Intelligence para las Finanzas
**Magister en Finanzas**

**Profesor: David Díaz **

**Autor: Álvaro Gutiérrez**

---

#Sesión #3:  Pandas 2.0 (_concat_ and _group_by_)


En esta sesión aprendermos lo básico sobre el manejo de bases de datos. Por ejemplo, cocatenar distintas bases de datos mediante [append](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html) y [concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html). Una vez hecho esto, haremos uso de group_by para trabajar con alguna submuestra de la data que cumpla con ciertas caracteristicas.

##Importando data a ocupar.

Se cargará un archivo excel que posee las 4 bases a utilizar en esta sesión. El archivo se cargará al ambiente Collaborative mediante el comando _!wget_ más el link de dropbox donde se encuentra almacenado. Adicionalmente, (posiblemente) necesitemos instalar la librería xlrd, la cual es la encargada de manejar archivos excel desde pandas. Esto se hará mediante el comando _!pip install xlrd_




In [93]:
!wget https://www.dropbox.com/s/3ch8lgbq65nu8fw/data_base_to_work_with.xlsx
!pip install xlrd  



Redirecting output to ‘wget-log.10’.


Este archivo contiene 4 hojas:


1.   **merge_firms**: Lista de firmas latinoamericanas con sus respectivos ratios financieros (EBIT, CAPEX, DEBT...). Información correspondiente al año 2014.
2.  **merge_firms_to_add**: Ratios adicionales de las mismas firmas listadas anteriormente, también corresponde a 2014.
3.  **merge_countries** : Información relacionada a ciertos países de la muestra (Tasa de inflacion, tasa impositiva, población total), también corresponde a 2014.
4. **append_2013**: Simil de la información de **merge_firms**, pero para año 2013.



In [94]:
import pandas as pd #Importando pandas
excel_file = 'data_base_to_work_with.xlsx' #Importando el archivo excel recién cargado mediante !wget
#Primera hoja del libro excel "merge_firms"
merge_firms = pd.read_excel(excel_file, sheetname='merge_firms')
merge_firms.head(3)

,id,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,...,at,dlt,dltt,dp,rd,dividend,stock,equity_share_holder,debtcap,age
0,ABEV3.SA,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-1.691000e+09,1.013250,15710000000,...,27150000000,607700000.0,NaN,NaN,NaN,NaN,15700000000,15900000000,0.058483,12.150580
1,AC.MX,Arca Continental SAB de CV,Mexico,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-3.119000e+08,1.939020,1611000000,...,5448000000,954800000.0,NaN,NaN,NaN,0.07634,1610000000,3120000000,0.255116,6.272416
2,AGR.BA,Agrometal SAI,Argentina,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014,-3.360144e+05,0.285851,48000000,...,27977816,3212402.5,NaN,41093.885,NaN,NaN,48000000,13700000,0.335568,59.373030


inspeccionando las dimesiones del objeto creado

In [95]:
merge_firms.shape

(191, 21)

##[Concat](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html): 

###Preproceso: set_index

Lo primero que realizaremos será pegar la segunda hoja del libro excel (**merge_firms_to_add**) con la base que acabamos de importar (**merge_firms**) haciendo ocupando como "llave" en el pegado a la columna **id**. Pero para esto necesitamos realizar un preproceso con el índice e incluir dicha columna como indice de nuestro DataFrame. Al activar la opción **inplace = True**,  permite sobre escribir el DataFrame modificado en el mismo que ya teníamos. En caso de activar False, entonces creará uno nuevo. Finalmente, activamos la opción **drop =True**, para no tener duplicado el index como variable también..  Luego repetimos el mismo procedimiento para la base a concatenar.




In [126]:
merge_firms.set_index('id', inplace=True, drop =True)#Reemplaza el indice por la informaciona contenida en la columna "id"
merge_firms.head(3)

,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,at,dlt,dltt,dp,rd,dividend,stock,equity_share_holder,debtcap,age
id,,,,,,,,,,,,,,,,,,,,
ABEV3.SA,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-1.691000e+09,1.013250,15710000000,9.871000e+08,27150000000,607700000.0,NaN,NaN,NaN,NaN,15700000000,15900000000,0.058483,12.150580
AC.MX,Arca Continental SAB de CV,Mexico,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-3.119000e+08,1.939020,1611000000,1.070000e+09,5448000000,954800000.0,NaN,NaN,NaN,0.07634,1610000000,3120000000,0.255116,6.272416
AGR.BA,Agrometal SAI,Argentina,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014,-3.360144e+05,0.285851,48000000,6.929635e+06,27977816,3212402.5,NaN,41093.885,NaN,NaN,48000000,13700000,0.335568,59.373030


ah

In [97]:
merge_firms.shape #Confirmamos que perdemos una columna del DataFrame

(191, 20)

**...Repitiendo para el segundo dataframe**

In [127]:
merge_firms_to_add = pd.read_excel(excel_file, sheetname='merge_firms_to_add')
merge_firms_to_add.set_index('id', inplace=True , drop =True) #Reemplaza el indice por la informaciona contenida en la columna "id"
merge_firms_to_add.head(5)

,total_equity,ebit,total_liability,revenue,size
id,,,,,
ABEV3.SA,1.589000e+10,6.016000e+09,1.126000e+10,1.433000e+10,23.38575
AC.MX,3.124000e+09,7.315000e+08,2.324000e+09,4.202000e+09,22.15887
AGR.BA,1.372086e+07,4.457442e+06,1.425696e+07,3.141261e+07,17.26272
AGU.SN,1.643000e+08,2.735000e+07,4.171000e+08,6.122000e+08,20.23258
AGUAa.SN,1.007000e+09,3.419000e+08,1.642000e+09,7.268000e+08,20.40417


In [99]:
merge_firms_to_add.shape

(191, 5)

##Concat: Pegado usando index como "llave"

Ahora podemos realizar el pegado de ambos dataframes en uno nuevo. Esto se realizará con los siguientes argumentos para la función concat.
1.  Elementos a concatenar [nuestros dos data frame separados por una coma ] 
2.  Axis: 1 es un pegado ocupando como llave el index (nuestro caso). Un cero es ocupando el nombre de las columanas como llave (a desarrollar más adelante)
3.  El indice de pegado, en este caso nos servirá cualquiera de los dos DataFrame ya que posee exactamente el mismo index (mismo número de observaciones.)

In [100]:

merged_data_2014 = pd.concat([merge_firms, merge_firms_to_add], axis=1, join_axes=[merge_firms_to_add.index])
merged_data_2014.shape

(191, 25)

## Concat: Pegado usando una columna como "llave"

Ahora cargaremos la base **merge_countries** y adicionaremos esa información a la base recién creada (**merged_data_2014**).**merge_countries** como mencionamos anteriormente, contiene información a un nivel de agregación diferente al de las dos bases anteriores. Las bases anteriores tenían un nivel de agregación "micro", es decir podíamos identificar a cada una de las firmas. Ahora la información corresponde a una agregación a nivel "macro", ya que es información a nivel país. Esto último, implica que las filas de la base **merge_countries**, serán asignadas a más de una de las filas de la base **merged_data_2014**, debido a que por ejemplo, la inflación de Chile (que es una fila de **merge_countries**) será asignada a todas las firmas que estén ubicadas en Chile.

In [101]:
#Cargando base merge_countries
merge_countries = pd.read_excel(excel_file,sheetname="merge_countries")
print(merge_countries)

     country  inflation  tax_rate  population
0  Argentina   0.947232  0.254727    43132000
1     Brazil   0.052223  0.398399   204519000
2      Chile   0.974318  0.248184    18006000
3   Colombia   0.945748  0.255085    48218000
4     Mexico   0.185648  0.392126   121006000


### Pegado "one to many" (como NO hacerlo....)

#### Primera forma de hacerlo MAL. No especificar el _axis_ del pegado.

In [102]:
merged_data_2014_countries_wrong1 =  pd.concat([merged_data_2014, merge_countries] , axis=1 )
merged_data_2014_countries_wrong1.head(8)

,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,...,age,total_equity,ebit,total_liability,revenue,size,country,inflation,tax_rate,population
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Argentina,0.947232,0.254727,43132000.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Brazil,0.052223,0.398399,204519000.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Chile,0.974318,0.248184,18006000.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Colombia,0.945748,0.255085,48218000.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Mexico,0.185648,0.392126,121006000.0
ABEV3.SA,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014.0,-1.691000e+09,1.013250,1.571000e+10,9.871000e+08,...,12.150580,1.589000e+10,6.016000e+09,1.126000e+10,1.433000e+10,23.38575,NaN,NaN,NaN,NaN
AC.MX,Arca Continental SAB de CV,Mexico,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014.0,-3.119000e+08,1.939020,1.611000e+09,1.070000e+09,...,6.272416,3.124000e+09,7.315000e+08,2.324000e+09,4.202000e+09,22.15887,NaN,NaN,NaN,NaN
AGR.BA,Agrometal SAI,Argentina,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014.0,-3.360144e+05,0.285851,4.800000e+07,6.929635e+06,...,59.373030,1.372086e+07,4.457442e+06,1.425696e+07,3.141261e+07,17.26272,NaN,NaN,NaN,NaN


#### Segunda forma de hacerlo MAL: Creer que Python es más inteligente que tu.

Si ocupamos el mismo delimitador de axis que ocupamos antes, entonces python encontrará cero coincidencias ya que nuestra nueva variable posee un index que va del cero al cuatro, lo que no tiene coincidencia, obtiendo algo horrible como lo que sigue.

In [103]:
merged_data_2014_countries_wrong2 =  pd.concat([merged_data_2014, merge_countries] , axis=1 ,join_axes=[merge_firms_to_add.index] )
merged_data_2014_countries_wrong2.head(8)

,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,...,age,total_equity,ebit,total_liability,revenue,size,country,inflation,tax_rate,population
id,,,,,,,,,,,,,,,,,,,,,
ABEV3.SA,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-1.691000e+09,1.013250,15710000000,9.871000e+08,...,12.150580,1.589000e+10,6.016000e+09,1.126000e+10,1.433000e+10,23.38575,NaN,NaN,NaN,NaN
AC.MX,Arca Continental SAB de CV,Mexico,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-3.119000e+08,1.939020,1611000000,1.070000e+09,...,6.272416,3.124000e+09,7.315000e+08,2.324000e+09,4.202000e+09,22.15887,NaN,NaN,NaN,NaN
AGR.BA,Agrometal SAI,Argentina,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014,-3.360144e+05,0.285851,48000000,6.929635e+06,...,59.373030,1.372086e+07,4.457442e+06,1.425696e+07,3.141261e+07,17.26272,NaN,NaN,NaN,NaN
AGU.SN,Agencias Universales SA,Chile,Industrials,Transportation,Freight & Logistics Services,2014,-6.054200e+07,0.192198,855100000,2.309000e+08,...,27.775500,1.643000e+08,2.735000e+07,4.171000e+08,6.122000e+08,20.23258,NaN,NaN,NaN,NaN
AGUAa.SN,Aguas Andinas SA,Chile,Utilities,Utilities,Water & Related Utilities,2014,-1.356000e+08,0.164602,6119000000,1.245000e+09,...,28.240930,1.007000e+09,3.419000e+08,1.642000e+09,7.268000e+08,20.40417,NaN,NaN,NaN,NaN
ALFAA.MX,Alfa SAB de CV,Mexico,Industrials,Industrial Conglomerates,Industrial Conglomerates,2014,-1.937000e+09,0.730240,5135000000,6.234000e+09,...,50.403830,3.756000e+09,1.172000e+09,1.204000e+10,1.555000e+10,23.46714,NaN,NaN,NaN,NaN
ALI.LM,Alicorp SAA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,-1.133000e+08,0.819730,854600000,9.253000e+08,...,12.238190,7.005000e+08,1.902000e+08,1.591000e+09,2.109000e+09,21.46952,NaN,NaN,NaN,NaN
ALPA4.SA,Alpargatas SA,Brazil,Consumer Cyclicals,Cyclical Consumer Products,Textiles & Apparel,2014,-5.245691e+07,1.631650,460800000,1.935000e+08,...,110.414800,7.519000e+08,1.307000e+08,6.221000e+08,1.294000e+09,20.98069,NaN,NaN,NaN,NaN


#### Tercera forma de hacerlo MAL: Perdile que ocupe como eje de pegado en base a la columna "country"

Bueno intento...pero NO. Lo que obtendremos al realizar esto será algo como esto.




In [104]:
merged_data_2014_countries_wrong3 =  pd.concat([merged_data_2014, merge_countries] , axis=1 ,join_axes=[merged_data_2014['country']] )
merged_data_2014_countries_wrong3.head(8)


,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,...,age,total_equity,ebit,total_liability,revenue,size,country,inflation,tax_rate,population
country,,,,,,,,,,,,,,,,,,,,,
Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Chile,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mexico,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Peru,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Brazil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Bueno, vemos que este último intento también salió pésimo... ¿Qué hemos aprendido de todo esto? Concat no nos servirá para lo que queremos hacer. Por ende recurriremos al próximo comando de esta sesión

# Solución?: [Merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html)!

Este otro comando nos permite indicarle que realize el pegado basado en alguna columna común que tengan ambos DataFrame. La que en este caso corresponde a la columna "**country**". La sintaxis de este comando corresponde a:

1. DataFrames a ser pegados: En nuestro caso (**merged_data_2014**, **merge_countries**)
2. Columna común que comparten ambos DataFrame que servirá de "llave"
3. how: Corresponde al orden de las variables 



In [105]:
merged_data_2014_countries = pd.merge(merged_data_2014, merge_countries, on='country')
merged_data_2014_countries.head(5)


,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,...,debtcap,age,total_equity,ebit,total_liability,revenue,size,inflation,tax_rate,population
0,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-1.691000e+09,1.013250,15710000000,987100000.0,...,0.058483,12.15058,1.589000e+10,6.016000e+09,1.126000e+10,1.433000e+10,23.38575,0.052223,0.398399,204519000
1,Alpargatas SA,Brazil,Consumer Cyclicals,Cyclical Consumer Products,Textiles & Apparel,2014,-5.245691e+07,1.631650,460800000,193500000.0,...,0.204641,110.41480,7.519000e+08,1.307000e+08,6.221000e+08,1.294000e+09,20.98069,0.052223,0.398399,204519000
2,Baumer SA,Brazil,Healthcare,Healthcare Services & Equipment,Healthcare Equipment & Supplies,2014,-1.658261e+06,2.834907,9800000,8060971.0,...,0.224896,65.31964,2.778208e+07,6.486263e+06,2.373767e+07,4.377004e+07,17.59446,0.052223,0.398399,204519000
3,Bardella SA Industrias Mecanicas,Brazil,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014,-2.783967e+06,79.303020,1600000,62066616.0,...,0.328479,106.55440,1.269000e+08,4.010915e+06,1.704000e+08,1.803000e+08,19.00988,0.052223,0.398399,204519000
4,BICICLETAS MONARK SA,Brazil,Consumer Cyclicals,Cyclical Consumer Products,Leisure Products,2014,-2.860369e+04,148.955030,454750,0.0,...,0.000000,69.34976,6.773730e+07,1.757621e+05,9.463681e+06,1.228754e+07,16.32410,0.052223,0.398399,204519000


####¿Dónde se fue Perú?

Si revisamos el DataFrame que recién creamos vemos que perú ha desaparecido de la base.

In [106]:
merged_data_2014_countries['country'].value_counts()


Brazil       62
Chile        58
Mexico       26
Argentina    23
Colombia      2
Name: country, dtype: int64

#### Recuperemos a Perú!

Qué ha pasado entonces? Lo que ha ocurrido es que hemos sido demasiado descuidados con un elemento clave del comando merge que es el atributo **HOW**. El cual nos indica la forma en la que realizaremos el pegado. En este caso este atributo tiene 4 opciones.

1. outer: Nos permite quedarnos con la intersección de ambos DataFrames. Conservando todos los valores que no tengan correspondencia en alguno de los DataFrame.
2. Inner: Genera un unión que intersecta los valores de ambos DataFrames. Es decir elimina cualquier valor no matcheado en alguno de los DataFrame. En este caso, Perú no aparece en la base **merge_countries**, por lo que se eliminará de la base. Esta es la opción por defecto de Pandas así que por Perú había desaparecido de la base.
3. left: Se queda sólo con los datos del lado izquierdo (primera base que se ingresa en los argumentos de **merge**)
4. right:  Se queda sólo con los datos del lado derecho (segunda base que se ingresa en los argumentos de **merge**)

> Si no recuerdas bien que significa "intersección", "union", "unión por la derecha o izquiera", puedes revisar el siguiente [link](https://en.wikipedia.org/wiki/Venn_diagram)

Por lo tanto... para recuperar a Perú podemos ocupar el siguiente comando






In [107]:
merged_data_2014_countries = pd.merge(merged_data_2014, merge_countries, on='country' , how = 'outer')
merged_data_2014_countries['country'].value_counts()


Brazil       62
Chile        58
Mexico       26
Argentina    23
Peru         20
Colombia      2
Name: country, dtype: int64

In [108]:
#Podemos ver que Perú se quedó en la base, pero efectivamente sin la información adicional de la base 'merge_country'
merged_data_2014_countries[merged_data_2014_countries['country']=='Peru' ].head(5)

,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,deb_tot,...,debtcap,age,total_equity,ebit,total_liability,revenue,size,inflation,tax_rate,population
169,Alicorp SAA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,-113300000.0,0.819730,854600000,9.253000e+08,...,0.569132,12.23819,7.005000e+08,190200000.0,1.591000e+09,2.109000e+09,21.46952,NaN,NaN,NaN
170,Corporacion Aceros Arequipa SA,Peru,Basic Materials,Mineral Resources,Metals & Mining,2014,-18970460.0,0.508297,1117000000,3.374000e+08,...,0.372631,19.58932,5.680000e+08,60305807.0,4.847000e+08,8.067000e+08,20.50852,NaN,NaN,NaN
171,Austral Group SAA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,-23395099.0,0.677967,259000000,1.191000e+08,...,0.404057,18.91855,1.756000e+08,6103726.1,1.675000e+08,1.526000e+08,18.84310,NaN,NaN,NaN
172,Compania de Minas Buenaventura SAA,Peru,Basic Materials,Mineral Resources,Metals & Mining,2014,-227600000.0,13.628182,254200000,4.463000e+08,...,0.114138,57.98494,3.464000e+09,61596000.0,1.208000e+09,1.176000e+09,20.88520,NaN,NaN,NaN
173,Central Azucarera Chucarapi Pampa Blanca SA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,0.0,8.133921,2342500,1.725411e+05,...,0.008974,24.29295,1.905371e+07,-2942742.5,1.341490e+07,4.704599e+06,15.36405,NaN,NaN,NaN


### Un Último Comentario.
Algo muy importante que un alumno juicioso debería haber notado a esta altura, es que hemos perdido el **index** de nuestro DataFrame, lo que lo ha vuelto completamente inútil ya que ha perdido su "llave". (con lo que básicamente obtuvimos  una pizza sin queso...)

**¿Cómo recuperarlo?**

Bueno, volvamos un poco en el tiempo, hasta la línea.


```
merge_firms.set_index('id', inplace=True , drop =True)
```

En esta linea hicimos que la columna llamada 'id' desapareciera del DataFrame para formar parte del 'index'. Ahora, adivinen... drop =False. Con eso mantendremos un respaldo del index dentro del DataFrame.
Ejecutemos esta versión paralela de nuestro código.

In [141]:
#Re-cargando merge_firms
merge_firms = pd.read_excel(excel_file, sheetname='merge_firms')
merge_firms=merge_firms.set_index('id', inplace=False ,drop= False) #Mantenemos columna id

#Re-cargando merge_firms_to_add
merge_firms_to_add = pd.read_excel(excel_file, sheetname='merge_firms_to_add')
merge_firms_to_add=merge_firms_to_add.set_index('id', inplace=False , drop =True) #Borramos columna id para que no esté duplicada

#Concat merge_firms y  merge_firms_to_add
merged_data_2014 = pd.concat([merge_firms, merge_firms_to_add], axis=1, join_axes=[merge_firms_to_add.index])

#Merge merged_data_2014 con merge_countries
merged_data_2014_countries = pd.merge(merged_data_2014, merge_countries, on='country' , how = 'outer')
merged_data_2014_countries[merged_data_2014_countries['country']=='Peru' ].head(5)


,id,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,...,debtcap,age,total_equity,ebit,total_liability,revenue,size,inflation,tax_rate,population
169,ALI.LM,Alicorp SAA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,-113300000.0,0.819730,854600000,...,0.569132,12.23819,7.005000e+08,190200000.0,1.591000e+09,2.109000e+09,21.46952,NaN,NaN,NaN
170,ARE.LM,Corporacion Aceros Arequipa SA,Peru,Basic Materials,Mineral Resources,Metals & Mining,2014,-18970460.0,0.508297,1117000000,...,0.372631,19.58932,5.680000e+08,60305807.0,4.847000e+08,8.067000e+08,20.50852,NaN,NaN,NaN
171,AUG.LM,Austral Group SAA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,-23395099.0,0.677967,259000000,...,0.404057,18.91855,1.756000e+08,6103726.1,1.675000e+08,1.526000e+08,18.84310,NaN,NaN,NaN
172,BUEv.LM,Compania de Minas Buenaventura SAA,Peru,Basic Materials,Mineral Resources,Metals & Mining,2014,-227600000.0,13.628182,254200000,...,0.114138,57.98494,3.464000e+09,61596000.0,1.208000e+09,1.176000e+09,20.88520,NaN,NaN,NaN
173,CHU.LM,Central Azucarera Chucarapi Pampa Blanca SA,Peru,Consumer Non-Cyclicals,Food & Beverages,Food & Tobacco,2014,0.0,8.133921,2342500,...,0.008974,24.29295,1.905371e+07,-2942742.5,1.341490e+07,4.704599e+06,15.36405,NaN,NaN,NaN


Ahora sólo nos falta repetir el proceso para asignar la columna 'id' como index al DataFrame recién creado.

In [142]:
merged_data_2014_countries= merged_data_2014_countries.set_index('id', inplace=False ,drop= False)
merged_data_2014_countries.head(5)

,id,CompanyCommonName,country,TRBCEconomicSectorName,TRBCBusinessSectorName,industry,time,capx,book_value_per_share,csho,...,debtcap,age,total_equity,ebit,total_liability,revenue,size,inflation,tax_rate,population
id,,,,,,,,,,,,,,,,,,,,,
ABEV3.SA,ABEV3.SA,Ambev SA,Brazil,Consumer Non-Cyclicals,Food & Beverages,Beverages,2014,-1.691000e+09,1.013250,15710000000,...,0.058483,12.15058,1.589000e+10,6.016000e+09,1.126000e+10,1.433000e+10,23.38575,0.052223,0.398399,204519000.0
ALPA4.SA,ALPA4.SA,Alpargatas SA,Brazil,Consumer Cyclicals,Cyclical Consumer Products,Textiles & Apparel,2014,-5.245691e+07,1.631650,460800000,...,0.204641,110.41480,7.519000e+08,1.307000e+08,6.221000e+08,1.294000e+09,20.98069,0.052223,0.398399,204519000.0
BALM4.SA,BALM4.SA,Baumer SA,Brazil,Healthcare,Healthcare Services & Equipment,Healthcare Equipment & Supplies,2014,-1.658261e+06,2.834907,9800000,...,0.224896,65.31964,2.778208e+07,6.486263e+06,2.373767e+07,4.377004e+07,17.59446,0.052223,0.398399,204519000.0
BDLL4.SA,BDLL4.SA,Bardella SA Industrias Mecanicas,Brazil,Industrials,Industrial Goods,"Machinery, Tools, Heavy Vehicles, Trains & Ships",2014,-2.783967e+06,79.303020,1600000,...,0.328479,106.55440,1.269000e+08,4.010915e+06,1.704000e+08,1.803000e+08,19.00988,0.052223,0.398399,204519000.0
BMKS3.SA,BMKS3.SA,BICICLETAS MONARK SA,Brazil,Consumer Cyclicals,Cyclical Consumer Products,Leisure Products,2014,-2.860369e+04,148.955030,454750,...,0.000000,69.34976,6.773730e+07,1.757621e+05,9.463681e+06,1.228754e+07,16.32410,0.052223,0.398399,204519000.0


Con lo que hemos pegado apropiadamente la información a nivel país, sin perder la información del identificador de cada empresa (**id**).


#GroupBy

Esto nos permitirá a acceder a niveles de agregación jerarquizados. Por ejemplo, en nuestra base de datos tenemos los siguientes niveles de agregación desde el más "micro" hasta el más "macro".
1. **Index** (columna **id** en un principio): Información a nivel de **firma**.
2. **TRBCEconomicSectorName**	: Información a nivel de **Sector Económico **.
3. **industry**: Información a nivel de **Industria**.
4. **country**: Información a nivel de **país**.

Por lo tanto, una serie de preguntas secillas que pueden ser respondida haciendo uso de este comando podrían ser:

In [144]:
#Promedio de activos totales(at) por industria.
merged_data_2014_countries.groupby('industry')['at'].mean()

industry
Aerospace & Defense                                 5.389350e+09
Automobiles & Auto Parts                            2.367000e+09
Beverages                                           9.132333e+09
Chemicals                                           3.273703e+09
Communications & Networking                         2.602000e+08
Computers, Phones & Household Electronics           2.379000e+08
Construction & Engineering                          2.418420e+09
Construction Materials                              7.482080e+09
Containers & Packaging                              8.616349e+07
Diversified Retail                                  8.535240e+09
Electric Utilities & IPPs                           7.458549e+09
Food & Drug Retailing                               4.408786e+09
Food & Tobacco                                      1.138845e+09
Freight & Logistics Services                        9.441833e+08
Healthcare Equipment & Supplies                     5.151976e+07
Healthcare Provi

In [148]:
#Mediana de la edad de las firmas (age) por país
merged_data_2014_countries.groupby('country')['age'].median()

country
Argentina    59.373030
Brazil       60.562630
Chile        50.599590
Colombia     37.478440
Mexico       30.837780
Peru         24.365505
Name: age, dtype: float64

In [154]:
#Promedio de la deuda total(total_liability) por país e industria
merged_data_2014_countries.groupby(['country', 'industry'])['total_liability'].mean()

country    industry                                        
Argentina  Chemicals                                           2.092844e+08
           Computers, Phones & Household Electronics           2.752000e+08
           Electric Utilities & IPPs                           2.782333e+08
           Food & Drug Retailing                               1.330800e+09
           Food & Tobacco                                      2.247814e+08
           Machinery, Tools, Heavy Vehicles, Trains & Ships    1.425696e+07
           Metals & Mining                                     5.555500e+08
           Natural Gas Utilities                               3.531000e+08
           Oil & Gas                                           5.492767e+09
           Paper & Forest Products                             1.273490e+07
           Pharmaceuticals                                     8.984983e+06
           Telecommunications Services                         1.393000e+09
           Textiles & Appare

In [157]:
merged_data_2014_countries.groupby(['country', 'industry'])['total_liability'].mean().reset_index()

,country,industry,total_liability
0,Argentina,Chemicals,2.092844e+08
1,Argentina,"Computers, Phones & Household Electronics",2.752000e+08
2,Argentina,Electric Utilities & IPPs,2.782333e+08
3,Argentina,Food & Drug Retailing,1.330800e+09
4,Argentina,Food & Tobacco,2.247814e+08
5,Argentina,"Machinery, Tools, Heavy Vehicles, Trains & Ships",1.425696e+07
6,Argentina,Metals & Mining,5.555500e+08
7,Argentina,Natural Gas Utilities,3.531000e+08
8,Argentina,Oil & Gas,5.492767e+09
9,Argentina,Paper & Forest Products,1.273490e+07
